In [1]:
import math 
import random 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import pandas_datareader as data_reader 
from tqdm import tqdm_notebook, tqdm 
from collections import deque

In [2]:
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
class AI_Trader():
  def __init__(self,state_size, action_space=3, model_name = "AITrader"):
    self.state_size=state_size
    self.action_space=action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name=model_name
    self.model = self.model_builder()

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
  def model_builder (self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    return model

  def trade (self, state):
   if random.random() <= self.epsilon:
    return random.randrange(self.action_space)
   actions = self.model.predict(state)

  def batch_train(self, batch_size):
   batch =[]
   for i in range(len(self.memory)-batch_size + 1, len(self.memory)):
     batch.append(self.memory[i])

   for state, action, reward, next_state, done in batch:
    reward = reward
    if not done:
      reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
    
    target = self.model.predict(state)
    target[0][action]=reward

    self.model.fit(state, target, epochs=1, verbose=0)

   if self.epsilon > self.epsilon_final:
     self. epsilon *= self.epsilon_decay

In [3]:
def sigmoid(x):
  return 1/(1+math.exp(-x))

In [4]:
# Формат вывода данных
def stock_price_format (n):
  if n<0:
    return "- # {0:2f}".format (abs(n))
  else:
    return "$ {0:2f}".format (abs(n))

In [5]:
# Загрузка цены закрытия актива из Yahoo Finance
def dataset_loader (stock_name):

  dataset = data_reader.DataReader(stock_name, data_source="yahoo")

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[1]).split()[0]

  close = dataset['Close']

  return close

In [9]:
dataset = data_reader.DataReader("TSLA", data_source="yahoo")
dataset

UnboundLocalError: ignored

In [6]:
import pandas
from pandas_datareader import data as pdr
import yfinance as yfin


yfin.pdr_override()

gold = pdr.get_data_yahoo('Gold', start='2021-10-24', end='2022-12-23')

print(gold)

[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2021-10-25  19.760000  19.770000  19.520000  19.549999  18.504694  12533000
2021-10-26  19.459999  19.580000  19.299999  19.430000  18.391111  12052200
2021-10-27  19.370001  19.530001  19.299999  19.389999  18.353252  10707500
2021-10-28  19.420000  19.469999  18.900000  18.940001  17.927313  20967400
2021-10-29  18.629999  18.719999  18.370001  18.370001  17.387791  22767500
...               ...        ...        ...        ...        ...       ...
2022-12-16  16.639999  17.059999  16.559999  16.900000  16.794374  19723800
2022-12-19  16.930000  16.980000  16.469999  16.549999  16.446562  16413900
2022-12-20  16.850000  17.410000  16.820000  17.280001  17.172001  25819900
2022-12-21  17.400000  17.670000  17.330000  17.500000  17.390625  20232900
2022-12-22  17.2300

In [7]:
def state_creator (data, timestep, window_size):
  starting_id = timestep - window_size +1
  if starting_id >= 0:
    windowed_data = data[starting_id: timestep+1]
  else:
    windowed_data = starting_id * [data[0]] + list(data[0:timestep+1])
  state = []
  for i in range(window_size -1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
  return np.array([state])


In [8]:
stock_name="Gold"
data = dataset_loader(stock_name)
data

UnboundLocalError: ignored

In [ ]:
window_size = 10
episodes = 1

batch_size = 32
data_samples = len(data) - 1000

In [ ]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 32)                352       
                                                                 
 dense_13 (Dense)            (None, 64)                2112      
                                                                 
 dense_14 (Dense)            (None, 128)               8320      
                                                                 
 dense_15 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for episode in range(1, episodes + 1):

  print ("Episode: {}/{}".format(episode, episodes))

  state = state_creator(data, window_size, window_size+1)

  total_profit = 0
  trader.inventory =[]
  for t in tqdm(range(window_size, data_samples)):

    action = trader.trade(state)

    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0

    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stock_price_format(data[t]))

    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print ("AI Trader sold: ", stock_price_format(data[t]), "Profit: " + stock_price_format(data[t] - buy_price) )

    if t == data_samples -1:
      done = True
    else:
      done = False
    
    trader.memory.append((state, action, reward, next_state, done))

    state = next_state

    if done:
      print("###############")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("##############")

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
    
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

Episode: 1/1


  0%|          | 0/247 [00:00<?, ?it/s]

AI Trader bought:  $ 20.582666
AI Trader sold:  $ 21.187332 Profit: $ 0.604666
AI Trader bought:  $ 20.590000
AI Trader sold:  $ 20.435333 Profit: - # 0.154667
AI Trader bought:  $ 20.346666
AI Trader bought:  $ 20.246668
AI Trader sold:  $ 20.884001 Profit: $ 0.537334
AI Trader sold:  $ 20.749332 Profit: $ 0.502665
AI Trader bought:  $ 22.896667
AI Trader sold:  $ 22.591333 Profit: - # 0.305334
AI Trader bought:  $ 22.073334
AI Trader bought:  $ 21.931999
AI Trader bought:  $ 22.110666
AI Trader sold:  $ 21.680000 Profit: - # 0.393333
AI Trader sold:  $ 21.152666 Profit: - # 0.779333
AI Trader sold:  $ 20.775999 Profit: - # 1.334667
AI Trader bought:  $ 21.024000
AI Trader sold:  $ 20.756666 Profit: - # 0.267334
AI Trader bought:  $ 21.105333
1/1 [==============================] - 0s 17ms/step


 13%|█▎        | 33/247 [00:05<00:37,  5.70it/s]

AI Trader bought:  $ 22.246000
1/1 [==============================] - 0s 18ms/step


...

 99%|█████████▉| 245/247 [20:24<00:11,  5.96s/it]

AI Trader bought:  $ 23.367332
1/1 [==============================] - 0s 17ms/step


100%|█████████▉| 246/247 [20:31<00:06,  6.19s/it]

AI Trader sold:  $ 22.085333 Profit: $ 5.270000
###############
TOTAL PROFIT: 7.289335250854492
##############
1/1 [==============================] - 0s 16ms/step


100%|██████████| 247/247 [20:37<00:00,  5.01s/it]
